In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/final_csv/final_model_df.csv')

In [ ]:
# Replace "Unknown" in Lon and Lat with 0.0
df['GPS (Longitude)'].replace(to_replace="Unknown",value=0.0,inplace=True)
df[' GPS (Latitude)'].replace(to_replace="Unknown",value=0.0,inplace=True)

In [ ]:
df[['GPS (Longitude)',' GPS (Latitude)']] = df[['GPS (Longitude)',' GPS (Latitude)']].astype(np.float64)

In [ ]:
# Too many unknowns in number of individuals served, so we drop it 
df.drop(columns=['Individuals Directly Served'],inplace=True)

In [ ]:
# Change assessment date to datetime format
df['Assessment Date'] = pd.to_datetime(df['Assessment Date'],infer_datetime_format=True)

In [ ]:
# remove commas from population 
df[' Population of Sector '] = df[' Population of Sector '].replace(',','',regex=True)

In [ ]:
df[' Population of Sector '] = df[' Population of Sector '].astype(np.float64).astype(np.int64)

## Format financial institutions data as 'minute' in float dtype

In [ ]:
df['Umurenge SACCO-2'] = df['Umurenge SACCO'].str.split(':').str[1]
df['Umurenge SACCO-2'] = df['Umurenge SACCO-2'].astype(np.float64)/60

df['Umurenge SACCO'] = df['Umurenge SACCO'].str.split(':').str[0]
df['Umurenge SACCO'] = df['Umurenge SACCO'].astype(np.float64)

df['Umurenge SACCO'] = df['Umurenge SACCO'] + df['Umurenge SACCO-2']

In [ ]:
df['MFIs-2'] = df['MFIs'].str.split(':').str[1]
df['MFIs-2'] = df['MFIs-2'].astype(np.float64)/60

df['MFIs'] = df['MFIs'].str.split(':').str[0]
df['MFIs'] = df['MFIs'].astype(np.float64)

df['MFIs'] = df['MFIs'] + df['MFIs-2']

In [ ]:
df['Bank Branch-2'] = df['Bank Branch'].str.split(':').str[1]
df['Bank Branch-2'] = df['Bank Branch-2'].astype(np.float64)/60

df['Bank Branch'] = df['Bank Branch'].str.split(':').str[0]
df['Bank Branch'] = df['Bank Branch'].astype(np.float64)

df['Bank Branch'] = df['Bank Branch'] + df['Bank Branch-2']

In [ ]:
df['ATM-2'] = df['ATM'].str.split(':').str[1]
df['ATM-2'] = df['ATM-2'].astype(np.float64)/60

df['ATM'] = df['ATM'].str.split(':').str[0]
df['ATM'] = df['ATM'].astype(np.float64)

df['ATM'] = df['ATM'] + df['ATM-2']

In [ ]:
df['Bank Agent-2'] = df['Bank Agent'].str.split(':').str[1]
df['Bank Agent-2'] = df['Bank Agent-2'].astype(np.float64)/60

df['Bank Agent'] = df['Bank Agent'].str.split(':').str[0]
df['Bank Agent'] = df['Bank Agent'].astype(np.float64)

df['Bank Agent'] = df['Bank Agent'] + df['Bank Agent-2']

In [ ]:
df['Mobile Money Agent-2'] = df['Mobile Money Agent'].str.split(':').str[1]
df['Mobile Money Agent-2'] = df['Mobile Money Agent-2'].astype(np.float64)/60

df['Mobile Money Agent'] = df['Mobile Money Agent'].str.split(':').str[0]
df['Mobile Money Agent'] = df['Mobile Money Agent'].astype(np.float64)

df['Mobile Money Agent'] = df['Mobile Money Agent'] + df['Mobile Money Agent-2']

In [ ]:
df.drop(columns=['Bridge Site Name','Assessment Date','Project Code',
                 'Mobile Money Agent-2','Bank Agent-2','ATM-2','Bank Branch-2','MFIs-2','Umurenge SACCO-2',
                 'Original_Community_col','Community_Served','Prov_ID','Dist_ID','Sect_ID','Sector'],inplace=True)

In [ ]:
df['Bridge Type'].value_counts()

Suspension    633
Suspended     596
Unknown        97
Other          21
Name: Bridge Type, dtype: int64

In [ ]:
df = pd.get_dummies(df,columns=['Bridge Type'])

In [ ]:
df = pd.get_dummies(df,columns=['Province'])

In [ ]:
df = pd.get_dummies(df,columns=['District'])

In [ ]:
df['Project Stage'].value_counts()

Rejected              1066
Identified             224
Complete                30
Prospecting             14
Under Construction       7
Confirmed                6
Name: Project Stage, dtype: int64

In [ ]:
# helper function to add under construction and confirmed together 
def code_ps(x):
  if x == 'Under Construction' or x == 'Confirmed'or x == 'Identified':
    x = 'Complete'
  return x

In [ ]:
df['Project Stage'] = df['Project Stage'].apply(code_ps)

In [ ]:
df['Project Stage'].value_counts()

Rejected       1066
Complete        267
Prospecting      14
Name: Project Stage, dtype: int64

In [ ]:
df = df.drop(df[df['Project Stage'] == 'Prospecting'].index)

In [ ]:
# Drop outliers in long and latitude
df = df.drop(df[df[' GPS (Latitude)'] == 0.0].index)

In [ ]:
df = df.drop(df[df['GPS (Longitude)'] == 0.0].index)

In [ ]:
df['Project Stage'].value_counts()

Rejected    1064
Complete     267
Name: Project Stage, dtype: int64

In [ ]:
def change_ps(x):
  if x == 'Complete':
    x = 1
  elif x == 'Rejected':
    x = 0 
  return x

In [ ]:
df['Project Stage'] = df['Project Stage'].apply(change_ps)

In [ ]:
df['Project Stage'].value_counts()

0    1064
1     267
Name: Project Stage, dtype: int64

In [ ]:
df.to_csv('model_df.csv',index=False,header=None)

In [ ]:
df.shape

(1331, 57)